In [0]:
%run ../source_to_bronze/utils

In [0]:
db_name="employee_info"
table_name="dim_employee"
path=f"dbfs:/FileStore/assignments/assign_1/silver/{db_name}/{table_name}"
delta_df = read_delta(path)
display(delta_df)

employee_id,employee_name,department,country,salary,age
1,James,D101,IN,9000,25
2,Michel,D102,SA,8000,26
3,James son,D101,IN,10000,35
4,Robert,D103,MY,11000,34
5,Scott,D104,MA,6000,36
6,Gen,D105,JA,21345,24
7,John,D102,MY,87654,40
8,Maria,D105,SA,38144,38
9,Soffy,D103,IN,23456,29
10,Amy,D103,CN,21345,24


In [0]:
display(find_sal_dep(delta_df))

employee_id,employee_name,department,country,salary,age,rank
1,James,D101,IN,9000,25,1
3,James son,D101,IN,10000,35,2
2,Michel,D102,SA,8000,26,1
7,John,D102,MY,87654,40,2
4,Robert,D103,MY,11000,34,1
10,Amy,D103,CN,21345,24,2
9,Soffy,D103,IN,23456,29,3
5,Scott,D104,MA,6000,36,1
6,Gen,D105,JA,21345,24,1
8,Maria,D105,SA,38144,38,2


In [0]:
display(delta_df.groupBy("country").count())

country,count
CN,1
SA,2
MY,2
IN,3
MA,1
JA,1


In [0]:
country_path_read='dbfs:/FileStore/assignments/assign_1/source_to_bronze/country_df.csv/part-00000-tid-2429375358653570560-a09a881c-cda1-4428-be38-29a720a67fe8-317-1-c000.csv'
country_schema=StructType([
    StructField("countryCode",StringType(),True),
    StructField("countryName",StringType(),True)
])
department_path_read='dbfs:/FileStore/assignments/assign_1/source_to_bronze/department_df.csv/part-00000-tid-826393801858667356-6bf8e7e8-db95-4ea3-978c-de0c431bcb79-318-1-c000.csv'
department_schema=StructType([
    StructField("departmentId",StringType(),True),
    StructField("departmentName",StringType(),True)
])
employee_path='dbfs:/FileStore/assignments/assign_1/source_to_bronze/employee_df.csv/part-00000-tid-8958534801557983101-843e525b-d29c-4eeb-8f0d-87dcd37dad59-319-1-c000.csv'
employee_schema=StructType([
    StructField("employeeId",IntegerType(),True),
    StructField("employeeName",StringType(),True),
    StructField("department",StringType(),True), 
    StructField("country",StringType(),True),
    StructField("salary",IntegerType(),True),
    StructField("age",IntegerType(),True)
])


In [0]:
country_df=read_schema(country_path_read,country_schema)
department_df=read_schema(department_path_read,department_schema)
employee_df=read_schema(employee_path,employee_schema)


country_df.display()
department_df.display()
employee_df.display()

countryCode,countryName
CN,China
IN,India
SA,South Africa
JA,Japan
MY,Malaysia
MA,Morocco


departmentId,departmentName
D101,Sales
D102,Marketing
D103,Finance
D104,Support
D105,HR


employeeId,employeeName,department,country,salary,age
1,James,D101,IN,9000,25
2,Michel,D102,SA,8000,26
3,James son,D101,IN,10000,35
4,Robert,D103,MY,11000,34
5,Scott,D104,MA,6000,36
6,Gen,D105,JA,21345,24
7,John,D102,MY,87654,40
8,Maria,D105,SA,38144,38
9,Soffy,D103,IN,23456,29
10,Amy,D103,CN,21345,24


In [0]:
df1=df_joins(employee_df,country_df,department_df)
display(df1.orderBy('departmentName'))

departmentName,countryName
Finance,Malaysia
Finance,India
Finance,China
HR,Japan
HR,South Africa
Marketing,South Africa
Marketing,Malaysia
Sales,India
Sales,India
Support,Morocco


In [0]:
display(find_avg(delta_df,Window))

department,avg_age,age
D101,30.0,25
D101,30.0,35
D102,33.0,26
D102,33.0,40
D103,29.0,34
D103,29.0,29
D103,29.0,24
D104,36.0,36
D105,31.0,24
D105,31.0,38


In [0]:
col_name="at_load_data"
country_df_date=add_load_data(country_df,col_name)
department_df_date=add_load_data(department_df,col_name)
employee_df_date=add_load_data(employee_df,col_name)

country_df_date.display()
department_df_date.display()
employee_df_date.display()

countryCode,countryName,at_load_data
CN,China,2024-04-16
IN,India,2024-04-16
SA,South Africa,2024-04-16
JA,Japan,2024-04-16
MY,Malaysia,2024-04-16
MA,Morocco,2024-04-16


departmentId,departmentName,at_load_data
D101,Sales,2024-04-16
D102,Marketing,2024-04-16
D103,Finance,2024-04-16
D104,Support,2024-04-16
D105,HR,2024-04-16


employeeId,employeeName,department,country,salary,age,at_load_data
1,James,D101,IN,9000,25,2024-04-16
2,Michel,D102,SA,8000,26,2024-04-16
3,James son,D101,IN,10000,35,2024-04-16
4,Robert,D103,MY,11000,34,2024-04-16
5,Scott,D104,MA,6000,36,2024-04-16
6,Gen,D105,JA,21345,24,2024-04-16
7,John,D102,MY,87654,40,2024-04-16
8,Maria,D105,SA,38144,38,2024-04-16
9,Soffy,D103,IN,23456,29,2024-04-16
10,Amy,D103,CN,21345,24,2024-04-16


In [0]:
folder='employee'
table_name='fact_employee'
df_name=employee_df_date

path=f"dbfs:/FileStore/assignments/assign_1/gold/{folder}/{table_name}"

write_df_gold(df_name,path)
display(spark.read.format('delta').load(path)) 

employeeId,employeeName,department,country,salary,age,at_load_data
1,James,D101,IN,9000,25,2024-04-16
2,Michel,D102,SA,8000,26,2024-04-16
3,James son,D101,IN,10000,35,2024-04-16
4,Robert,D103,MY,11000,34,2024-04-16
5,Scott,D104,MA,6000,36,2024-04-16
6,Gen,D105,JA,21345,24,2024-04-16
7,John,D102,MY,87654,40,2024-04-16
8,Maria,D105,SA,38144,38,2024-04-16
9,Soffy,D103,IN,23456,29,2024-04-16
10,Amy,D103,CN,21345,24,2024-04-16


In [0]:
folder='country'
table_name='fact_country'
df_name=country_df_date

path=f"dbfs:/FileStore/assignments/assign_1/gold/{folder}/{table_name}"

write_df_gold(df_name,path)
display(spark.read.format('delta').load(path)) 

countryCode,countryName,at_load_data
CN,China,2024-04-16
IN,India,2024-04-16
SA,South Africa,2024-04-16
JA,Japan,2024-04-16
MY,Malaysia,2024-04-16
MA,Morocco,2024-04-16


In [0]:
folder='department'
table_name='fact_department'
df_name=department_df_date

path=f"dbfs:/FileStore/assignments/assign_1/gold/{folder}/{table_name}"

write_df_gold(df_name,path)
display(spark.read.format('delta').load(path)) 

departmentId,departmentName,at_load_data
D101,Sales,2024-04-16
D102,Marketing,2024-04-16
D103,Finance,2024-04-16
D104,Support,2024-04-16
D105,HR,2024-04-16
